In [1]:
import os
import re
import nltk
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras.models import load_model


train_data = pd.read_excel("/kaggle/input/nlp-project/train.xlsx")

train_data.info()  

train_data.columns = ['review_description','rating']
train_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32036 entries, 0 to 32035
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review_description  32036 non-null  object
 1   rating              32036 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 500.7+ KB


,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1


In [2]:
# here we make preprocessing on the data
import re

search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','"','?','؟','!']
replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
def process_review(review):
    processed_review = re.sub(r"[^\w\s]", '', review)# accept all things accept any word,digit,white space and replace with empty string
    processed_review = re.sub(r"[a-zA-Z]", '', processed_review) # here I remove all english lettters
    for i in range(0, len(search)):
        processed_review = processed_review.replace(search[i], replace[i])
    processed_review = re.sub(r"\d+", '', processed_review)
    processed_review = re.sub(r"\n", '', processed_review)# here remove new line
    processed_review = re.sub(r"\s+", ' ', processed_review)# here remove white spaces
    processed_review = re.sub(r'(.)\1+', r'\1', processed_review)# here if there are any repeate in any character replace it with one occurance
    
    return processed_review.strip() # return review after precessed without any additional white space

train_data['cleaned_text'] = train_data.review_description.apply(process_review)
# train_data = train_data[train_data.cleaned_text != ""]
train_data.head(10)
 

,review_description,rating,cleaned_text
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1,شركه زباله وسواقين بتبرشم ومفيش حتي رقم لشكاوي...
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1,خدمه الدفع عن طريق الكي نت توقفت عندي اصبح فقط...
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1,تطبيق غبي وجاري حذفه عاملين اكواد خصم ولما نست...
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1,فعلا تطبيق متاز بس لو في امكانيه يتيح لمستخدم ...
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1,سيء جدا اسعار رسوم التوصيل لا تمت لواقع ب صله
5,قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...,0,قعد عشرين سنه يدور علي سائق بس اما عن توصيل ال...
6,احلئ تطبيق,1,احلئ تطبيق
7,رائع واو مدهش,1,رائع واو مدهش
8,مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...,-1,مکو بس البحرین وعمان وغیره بس العراق مکو یعنی ...
9,تطبيق جميل يستاهل الخمس نجوم👍👍👍,1,تطبيق جميل يستاهل الخمس نجوم


In [3]:
import numpy as np
X = train_data.cleaned_text.values #training paramter
Y = train_data['rating'].astype('int') #prediction paramter
# Y = Y.clip(0, 1)


In [4]:
from nltk.tokenize import WordPunctTokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk import word_tokenize  
tokenizer = WordPunctTokenizer()
X_tokenized = [word_tokenize(sentence.lower()) for sentence in X]
word_index = {word: index + 1 for index, word in enumerate(set(word_tokenize(" ".join(X))))}
X_sequences = [[word_index.get(word, 0) for word in sentence] for sentence in X_tokenized]
maxlen = max(len(seq) for seq in X_sequences)
print(maxlen)
X_padded = pad_sequences(X_sequences, padding='post', maxlen=maxlen)
X=X_padded  
print(X)
print("Vocabulary size:", len(word_index))
print(X.shape[1])
# print(len(X))

362
[[10326  6131   226 ...     0     0     0]
 [20782  2596 19179 ...     0     0     0]
 [26136  2237 27874 ...     0     0     0]
 ...
 [29994 29479 26410 ...     0     0     0]
 [26136 26221 26396 ...     0     0     0]
 [29811 16509 20971 ...     0     0     0]]
Vocabulary size: 33372
362


In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y= label_encoder.fit_transform(Y)
print(Y)

[0 2 0 ... 0 0 2]


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, label_train, label_val = train_test_split(X , Y, test_size=0.4, random_state=42)
X_val,X_test,label_val,label_test = train_test_split( X_val , label_val, test_size=0.5, random_state=42)
print("Training:", len(X_train))
print("valadation",len(X_val))
print("Testing: ", len(X_test))

Training: 19221
valadation 6407
Testing:  6408


In [7]:
 
embedding_dim = 100
dropout = 0.5
opt = 'adam'
clear_session()
model = Sequential()
model.add(layers.Embedding(input_dim=33373, 
                           output_dim=362, 
                           input_length=362))
model.add(layers.Bidirectional(layers.LSTM(100, dropout=0.5, 
                                           recurrent_dropout=0.5, 
                                           return_sequences=True)))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer=opt, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 362, 362)          12081026  
                                                                 
 bidirectional (Bidirection  (None, 362, 200)          370400    
 al)                                                             
                                                                 
 global_max_pooling1d (Glob  (None, 200)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 128)               25728     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8

In [8]:
history = model.fit(X_train, label_train,
                    epochs=2,
                    verbose=True,
                    validation_data=( X_val, label_val),
                    batch_size=32)
loss, accuracy = model.evaluate(X_train, label_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss_val, accuracy_val = model.evaluate(X_test, label_test, verbose=True)
print("valadation Accuracy:  {:.4f}".format(accuracy_val))

Epoch 1/2
601/601 [==============================] - 928s 2s/step - loss: 0.6433 - accuracy: 0.7564 - val_loss: 0.4712 - val_accuracy: 0.8336
Epoch 2/2
601/601 [==============================] - 117s 194ms/step - loss: 0.3045 - accuracy: 0.8972
Training Accuracy: 0.8972
201/201 [==============================] - 38s 191ms/step - loss: 0.4695 - accuracy: 0.8311
valadation Accuracy:  0.8311


In [9]:
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = np.mean(predicted_labels == label_test)
print("Test Accuracy:", accuracy)

201/201 [==============================] - 39s 190ms/step
Test Accuracy: 0.8311485642946317


In [10]:
model.save('model_b32_e100.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
test_data=pd.read_csv("/kaggle/input/nlp-project/test _no_label.csv")
 
test_data['cleaned_text'] = test_data.review_description.apply(process_review)
# test_data = test_data[test_data.cleaned_text != ""]
# print(test_data.head(10))
 
reviews_test=test_data.cleaned_text.values
X_tokenized_test = [word_tokenize(sentence.lower()) for sentence in reviews_test]
print(len(X_tokenized_test))
X_sequences_test = [[word_index.get(word, 0) for word in sentence] for sentence in X_tokenized_test]
print(len(X_sequences_test))
maxlen = max(len(seq) for seq in X_sequences)
reviews_test = pad_sequences(X_sequences_test, padding='post', maxlen=maxlen)
print(reviews_test)
pred = model.predict(reviews_test,verbose=True)
pred = np.argmax(pred, axis=1)
print(pred)
pred = label_encoder.inverse_transform(pred)
print(pred) 

1000
1000
[[18730 11293 20782 ...     0     0     0]
 [ 9562 13126 18276 ...     0     0     0]
 [13218 29178  9329 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [26136  9759 24495 ...     0     0     0]
 [28428  6710  9759 ...     0     0     0]]
32/32 [==============================] - 6s 186ms/step
[2 2 2 2 2 2 0 0 2 0 2 2 2 0 0 2 2 0 0 0 2 2 2 2 0 0 2 2 2 0 2 0 0 0 0 2 2
 2 2 2 2 2 2 2 0 0 2 0 2 2 2 2 0 0 0 2 0 0 0 0 2 2 0 0 2 2 0 2 0 0 0 2 2 0
 0 0 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 2 2 0 0 0 0 2 2 0 0 2 2 2
 2 2 0 2 0 2 2 2 0 2 2 2 2 2 0 2 0 2 2 0 0 2 2 2 0 0 2 2 2 0 0 2 0 2 0 0 2
 2 2 2 2 2 0 0 0 2 0 2 2 2 2 2 2 0 0 2 2 2 2 2 0 2 2 0 2 2 0 2 0 0 2 2 2 2
 2 2 2 2 2 2 0 2 2 0 2 0 0 2 2 2 2 2 0 0 2 2 0 2 2 0 0 2 2 0 2 2 0 2 2 2 0
 0 2 0 2 0 0 0 2 2 2 2 2 2 2 0 0 2 0 0 0 0 2 0 2 2 0 0 2 2 0 0 0 0 2 0 2 2
 2 2 2 2 2 0 2 0 0 0 2 0 2 2 0 0 0 0 2 2 2 2 2 0 0 0 0 2 2 2 0 2 2 2 2 2 0
 2 2 2 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 2 2 0 0 2 0 2 2 2 2 2 0 0 

In [12]:
import csv
with open('output_predition.csv',mode='w')as out:
    field=['ID','rating']
    writer=csv.DictWriter(out,fieldnames=field)
    writer.writerow({'ID':'ID','rating':'rating'})
    for i in range(1,1001,1):
        writer.writerow({'ID':i,'rating':pred[i-1]})